# TutQ4 Quantum General Adverserial Network

This demo constructs a General Adverserial Network (GAN)
from a quantum circuit that serves as a generator and a second
quantum circuit which takes the role of a discriminator.

Quantum GANs could find application in the task of "unitary learning", which means to learn a quantum circuit that prepares a state given for example by a device with unknown dynamics.

### Imports

In [1]:
import pennylane as qml
from pennylane import numpy as np
np.random.seed(0)
dev = qml.device('default.qubit', wires=3)

### Classical and quantum nodes

Instead of drawing samples from a "true" distribution and the generator, the idea of the quantum GAN is to have a "true" circuit and a generator circuit prepare a quantum state, on which the circuit of the discriminator has to act as a classifier.

The "true circuit" is a simple rotation.

In [2]:
def true(phi, theta, omega):
    qml.Rot(phi, theta, omega, wires=0)

The generator and discriminator circuits are made up of the same basic routine.

In [3]:
def generator(w):
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=1)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=1)
    qml.CNOT(wires=[0,1])
    qml.RX(w[6], wires=0)
    qml.RY(w[7], wires=0)
    qml.RZ(w[8], wires=0)
    
def discriminator(w):
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=2)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=2)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=2)
    qml.CNOT(wires=[1,2])
    qml.RX(w[6], wires=2)
    qml.RY(w[7], wires=2)
    qml.RZ(w[8], wires=2)

This is how the generator and discriminator models are built:

In [4]:
@qml.qnode(dev)
def true_disc_circuit(phi, theta, omega, disc_weights):
    true(phi, theta, omega)
    discriminator(disc_weights)
    return qml.expval.PauliZ(2)

@qml.qnode(dev)
def gen_disc_circuit(gen_weights, disc_weights):
    generator(gen_weights)
    discriminator(disc_weights)
    return qml.expval.PauliZ(2)

### Cost

The cost is associated with the probability that the discriminator guesses "true" for a "fake" state (i.e. a state prepared by the generator instead of the discriminator).

In [5]:
def prob_real_true(disc_weights):
    true_disc_output = true_disc_circuit(phi, theta, omega, disc_weights)
    # convert to probability
    prob_real_true = (true_disc_output + 1) / 2
    return prob_real_true

def prob_fake_true(gen_weights, disc_weights):
    fake_disc_output = gen_disc_circuit(gen_weights, disc_weights)
    # convert to probability
    prob_fake_true = (fake_disc_output + 1) / 2
    return prob_fake_true # want to minimize this prob

def disc_cost(disc_weights):
    cost = prob_fake_true(gen_weights, disc_weights) - prob_real_true(disc_weights) 
    return cost

def gen_cost(gen_weights):
    return -prob_fake_true(gen_weights, disc_weights)

### Optimization

We initialize the fixed angles of the "true circuit", as well as some variables.

In [6]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
eps = 1e-2
gen_weights = np.array([0] + [np.pi] + [0] * 7) + eps * np.random.normal(size=[9])
disc_weights = np.random.normal(size=[9])

Creating an optimizer...

In [7]:
opt = qml.GradientDescentOptimizer(0.1)

..we first train the discriminator.

In [8]:
for it in range(50):
    disc_weights = opt.step(disc_cost, disc_weights) 
    cost = disc_cost(disc_weights)
    if it % 5 == 0:
        print("Step {}: cost = {}".format(it+1, cost))

Step 1: cost = -0.10942017805789189
Step 6: cost = -0.38998842264903155
Step 11: cost = -0.6660191175815633
Step 16: cost = -0.8550839212078478
Step 21: cost = -0.9454459581664489
Step 26: cost = -0.9805878247866402
Step 31: cost = -0.9931371328342751
Step 36: cost = -0.9974896764916592
Step 41: cost = -0.9989863506630718
Step 46: cost = -0.9995000463932011


The probability of the discriminator to be correct should be close to one at the discriminators optimum.

In [9]:
prob_real_true(disc_weights)

0.9998971951842259

Vice versa, the probability of the discriminator to be wrong should be close to zero at the discriminators optimum.

In [10]:
prob_fake_true(gen_weights, disc_weights)

0.00024278396179999717

In the adverserial game we have to now train the generator (and one can continue training the models in an alternating fashion).

In [11]:
for it in range(300):
    gen_weights = opt.step(gen_cost, gen_weights)
    cost = -gen_cost(gen_weights)
    if it % 5 == 0:
        print("Step {}: cost = {}".format(it, cost))

Step 0: cost = 0.0002664691382995854
Step 5: cost = 0.0004266200858930591
Step 10: cost = 0.0006872486146978218
Step 15: cost = 0.0011111626380135853
Step 20: cost = 0.0018000510248329382
Step 25: cost = 0.0029179304125440675
Step 30: cost = 0.004727717539773968
Step 35: cost = 0.007646628881031792
Step 40: cost = 0.012325866735736435
Step 45: cost = 0.019754518934527343
Step 50: cost = 0.03136834673567157
Step 55: cost = 0.049097345993078134
Step 60: cost = 0.07520378135265438
Step 65: cost = 0.11169015288702133
Step 70: cost = 0.15917286333740954
Step 75: cost = 0.2156603134394734
Step 80: cost = 0.2763735721045272
Step 85: cost = 0.33541691865274736
Step 90: cost = 0.3883501266928646
Step 95: cost = 0.43371772120148877
Step 100: cost = 0.4728490188392858
Step 105: cost = 0.5087778323625884
Step 110: cost = 0.5451977336157738
Step 115: cost = 0.585663291639799
Step 120: cost = 0.6327897835085339
Step 125: cost = 0.6872469221106772
Step 130: cost = 0.7468453348018596
Step 135: cost = 

At the optimum of the generator, the probabilities should be swapped, in other words, the probability of the discriminator to be fooled should be close to 1.

In [12]:
# should be close to one at G's optimum
prob_real_true(disc_weights)

0.9998971951842259

At the joint optimum the cost will be close to zero.

In [13]:
disc_cost(disc_weights)

-1.669397242665127e-08